# <pre>Building Dashboard For Restaurant

## Importing Necessary Packages, Libraries & Starting `pyngork`

In [2]:
#intall streamlit
!pip install streamlit -q
#To hold a temporary local server
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.6 MB/s eta 0:00:00


In [8]:
#obtaining external IP address
!wget -q -O - ipv4.icanhazip.com

34.19.55.119


In [9]:
# specifying the authorization token

from pyngrok import ngrok
!ngrok authtoken 2ms1mDrkTaNFMOKnok6Tg13iLZp_5x49SPxvirKskFtXgpeJm

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [10]:
# Starting the tunnel on port 8501 for HTTP traffic
url = ngrok.connect(8501, "http")
print(f'Access the Dashboard at : {url}')

Access the Dashboard at : NgrokTunnel: "https://1f76-34-19-55-119.ngrok-free.app" -> "http://localhost:8501"


## Writing the Streamlit Code

### 1. Setting the Page Configuration

In [13]:
# start the streamlit session
!streamlit run app.py &>/content/logs.txt &

In [257]:
%%writefile app.py

import os
import pandas as pd
import plotly.express as px
import streamlit as st

import warnings
warnings.filterwarnings('ignore')

#configuration
st.set_page_config(page_title = 'Zomato Dashboard', layout = 'wide')

# Set page title
st.title('Dashboard For Zomato Restaurants:bar_chart:')

#moving page title upwards and in center
st.markdown(
    '''<style>
            div.block-container{
              padding-top:2rem;
              text-align:center;
            }
       </style>
    ''',
    unsafe_allow_html = True)

Overwriting app.py


### 2. Upload the Data

In [258]:
%%writefile -a app.py

os.chdir(r'/content')
df = pd.read_csv('zomato.csv')

Appending to app.py


### 3. Creating Filters

SIDEBAR CREATION

In [259]:
%%writefile -a app.py

# 3.1 Sidebar for Restaurant Name
st.sidebar.header('Filters')
res_name = st.sidebar.multiselect('Restaurant Name', df['name'].unique())

if not res_name:
  df1 = df.copy()
else:
  df1 = df[df['name'].isin(res_name)]

# 3.2 Sidebar for restaurant type
res_type = st.sidebar.multiselect('Restaurant Type', df1['rest_type'].unique())

if not res_type:
  df2 = df1.copy()
else:
  df2 = df1[df1['rest_type'].isin(res_type)]

# 3.3 Sidebar for City
city = st.sidebar.multiselect('City', df2['listed_in(city)'].unique())

if not city:
  df3 = df2.copy()
else:
  df3 = df2[df2['listed_in(city)'].isin(city)]

Appending to app.py


LOGIC FOR SIDEBAR

In [260]:
%%writefile -a app.py

if not res_name and not res_type and not city:
  filters_df = df
elif not res_type and not city:
  filters_df = df1[df1['name'].isin(res_name)]
elif not res_name and not city:
  filters_df = df2[df2['rest_type'].isin(res_type)]
elif not res_name and not res_type:
  filters_df = df3[df3['listed_in(city)'].isin(city)]
elif res_name and res_type:
  filters_df = df3[df1['name'].isin(res_name) & df2['rest_type'].isin(res_type)]
elif res_name and city:
  filters_df = df3[df1['name'].isin(res_name) & df3['listed_in(city)'].isin(city)]
elif res_type and city:
  filters_df = df3[df2['rest_type'].isin(res_type) & df3['listed_in(city)'].isin(city)]
else:
  filters_df = df3[df1['name'].isin(res_name) & df2['rest_type'].isin(res_type) & df3['listed_in(city)'].isin(city)]

Appending to app.py


### 4. Cost fot two people by Restaurant Type and city

- `errors='coerce'` ensures that any problematic values are turned into NaN rather than raising an error.


In [261]:
%%writefile -a app.py

block1, block2 = st.columns((2))

filters_df['approx_cost(for two people)'] = pd.to_numeric(filters_df['approx_cost(for two people)'].str.replace(',', ''), errors='coerce')

# 1. Restaurant Type Wise Costs
cost_df = filters_df.groupby(by = 'rest_type', as_index = False)['approx_cost(for two people)'].mean()

with block1:
  st.subheader(' :moneybag: Restaurant Type Wise Cost')
  fig1 = px.bar(cost_df,
                x = 'rest_type',
                y = 'approx_cost(for two people)',
                template = 'seaborn')
  st.plotly_chart(fig1, use_container_width = True)

# 2. City Wise Costs
cost_df = filters_df.groupby(by = 'listed_in(city)', as_index = False)['approx_cost(for two people)'].mean()

with block2:
  st.subheader(' :moneybag: City Wise Costs')
  fig1 = px.pie(cost_df,
                values = 'approx_cost(for two people)',
                names = 'listed_in(city)',
                hole = 0.5)
  st.plotly_chart(fig1, use_container_width = True)

Appending to app.py


### 5. Correlation between cost and ratings

In [262]:
%%writefile -a app.py

st.subheader('Cost Vs Ratings')

fig3 = px.scatter(filters_df,
                  x='approx_cost(for two people)',
                  y='rate',
                  height=500,
                  width=1000,
                  template='gridon',
                  color='rate')

st.plotly_chart(fig3, use_container_width=True)


Appending to app.py


### 6. Pivot Table For Rwstaurant Name

In [263]:
%%writefile -a app.py

st.subheader('Data For Restaurants:page_with_curl:')
with st.expander("Restaurant Summary"):
  st.markdown('Restaurant Data')
  cols = (filters_df[['name', 'online_order', 'book_table', 'phone', 'address']]).astype('str')
  res_summary = pd.pivot_table(data = cols,
                               index = ['name'],
                               aggfunc=lambda x: ' '.join(x.unique()) if x.notnull().any() else '')
  st.write(res_summary.style.background_gradient(cmap = 'Blues'))

Appending to app.py


### 7. Votes fot two people by Restaurant Type and city

In [264]:
%%writefile -a app.py

block1, block2 = st.columns((2))

# filters_df['votes'] = pd.to_numeric(filters_df['votes'].str.replace(',', ''), errors='coerce')

# 1. Restaurant Type Wise Votes
cost_df = filters_df.groupby(by = 'rest_type', as_index = False)['votes'].mean()

with block1:
  st.subheader('Restaurant Type Wise Votes')
  fig1 = px.bar(cost_df,
                x = 'rest_type',
                y = 'votes',
                template = 'seaborn',
                color_discrete_sequence=['#9ACD32'])
  st.plotly_chart(fig1, use_container_width = True)

# 2. City Wise Votes
cost_df = filters_df.groupby(by = 'listed_in(city)', as_index = False)['votes'].mean()

with block2:
  st.subheader('City Wise Votes')
  fig2 = px.pie(cost_df,
                values = 'votes',
                names = 'listed_in(city)',
                hole = 0.5)
  st.plotly_chart(fig2, use_container_width = True)

Appending to app.py


### 8. Creating Treemap based on city, restaurant type and special food items they offer considering food cost

In [265]:
%%writefile -a app.py
st.subheader('Hierarchical View Restaurant Costs :arrow_down:')

data = filters_df.dropna(subset=['listed_in(city)', 'rest_type', 'listed_in(type)'])

mean_cost_df = data.groupby(['listed_in(city)', 'rest_type', 'listed_in(type)'], as_index=False)['approx_cost(for two people)'].mean()

fig4 = px.treemap(mean_cost_df,
                  path = ['listed_in(city)', 'rest_type', 'listed_in(type)'],
                  values = 'approx_cost(for two people)',
                  hover_data  = ['approx_cost(for two people)'],
                  color = 'rest_type')
fig3.update_layout(width = 800, height = 650)
st.plotly_chart(fig4, use_container_width = True)

Appending to app.py
